# Connecting to the Predictit 'API'

https://predictit.freshdesk.com/support/solutions/articles/12000001878-does-predictit-make-market-data-available-via-an-api-

In [1]:
import requests
import sys
import configparser
import datetime
import pytz

import pandas as pd
import tweepy

from bs4 import BeautifulSoup

sys.path.append("..")
from src.tweetdownloader import TweetDownloader
import src.utils as utils
import src.poisson as poisson

In [2]:
# we'll start with rdt
market_number = '6689'
url = f"https://www.predictit.org/api/marketdata/markets/{market_number}/"
url

'https://www.predictit.org/api/marketdata/markets/6689/'

In [3]:
response = requests.get(url)

In [4]:
response_data = response.json()

In [5]:
response_data

{'id': 6689,
 'name': 'How many tweets will @realDonaldTrump post from noon May 6 to 13?',
 'shortName': '@realDonaldTrump tweets 5/6 to 5/13?',
 'image': 'https://az620379.vo.msecnd.net/images/Markets/36231f71-30ba-47c1-8f5d-c7481e014c11.jpg',
 'url': 'https://www.predictit.org/markets/detail/6689/How-many-tweets-will-@realDonaldTrump-post-from-noon-May-6-to-13',
 'contracts': [{'id': 22581,
   'dateEnd': '2020-05-13T12:00:00',
   'image': 'https://az620379.vo.msecnd.net/images/Contracts/small_36231f71-30ba-47c1-8f5d-c7481e014c11.jpg',
   'name': '240 or more',
   'shortName': '240+',
   'status': 'Open',
   'lastTradePrice': 0.3,
   'bestBuyYesCost': 0.38,
   'bestBuyNoCost': 0.68,
   'bestSellYesCost': 0.32,
   'bestSellNoCost': 0.62,
   'lastClosePrice': 0.01,
   'displayOrder': 8},
  {'id': 22588,
   'dateEnd': '2020-05-13T12:00:00',
   'image': 'https://az620379.vo.msecnd.net/images/Contracts/small_36231f71-30ba-47c1-8f5d-c7481e014c11.jpg',
   'name': '210 - 219',
   'shortName':

## Construct buckets with their costs

In [6]:
title = response_data["contracts"][0]["name"]

In [7]:
def num_from_title(title): return [int(i) for i in title.split() if i.isdigit()]

In [39]:
lower = min([num_from_title(i["name"])[0] for i in response_data["contracts"]])

In [9]:
title = response_data["contracts"][1]["name"]
gaps = num_from_title(title)
gap = gaps[1] - gaps[0] + 1

## get the number of tweets since the start

yeah we're gonig to need to change this

In [10]:
end_date = pd.to_datetime(response_data["contracts"][0]["dateEnd"])
end_date = end_date.tz_localize("US/Eastern")


In [11]:
start_date = end_date - datetime.timedelta(days=7)
start_date

Timestamp('2020-05-06 12:00:00-0400', tz='US/Eastern')

In [12]:
time_left = end_date - datetime.datetime.now(tz=pytz.timezone('US/Eastern'))

In [13]:
datetime.datetime.now(tz=pytz.timezone('US/Eastern'))

datetime.datetime(2020, 5, 10, 16, 51, 23, 921644, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)

In [14]:
time_left.days

2

In [15]:
days_left = time_left.days + round((time_left.seconds / 2880) - (time_left.days * 2) / 24) * .01

In [16]:
rdt_tweets = TweetDownloader(to_load="realDonaldTrump")

INFO:root:API created
INFO:root:Liberating realDonaldTrump
INFO:root:	Page: 0
INFO:root:Found: 1 duplicate rows
INFO:root:Writing ../data/realDonaldTrump.csv


In [17]:
tweets = rdt_tweets.dataset

In [18]:
tweets_so_far = len(tweets[:start_date])

### TODO from here

Now we need to:

* [ ] make a bucket dit for the prices
* [ ] make kelly calcs

In [22]:
def process_name(name):
    numbers = num_from_title(name)
    if 'fewer' in name:
        return str(numbers[0])
    elif 'more' in name:
        return str(numbers[0])
    else:
        return str(numbers[0]) + '-' + str(numbers[1])

In [24]:
prices = dict()
for contract in response_data["contracts"]:
    prices[process_name(contract["name"])] = contract["bestBuyYesCost"]

In [25]:
prices

{'240': 0.38,
 '210-219': 0.18,
 '220-229': 0.16,
 '200-209': 0.15,
 '230-239': 0.15,
 '190-199': 0.1,
 '180-189': 0.05,
 '170-179': 0.02,
 '169': 0.01}

In [26]:
def setup_data(name, days=90):
    df = utils.tweets(name)
    by_day = df.groupby(df.index.date).count()
    cutoff_date = by_day.iloc[len(by_day)-1].name - datetime.timedelta(days=days)
    return by_day[cutoff_date:]

In [40]:
odds = poisson.basic_poisson(setup_data('realDonaldTrump')["ID"], days_left, tweets_so_far, lower=lower, gap=gap)

In [41]:
odds

{'169': 0.0,
 '170-179': 0.0,
 '180-189': 0.0,
 '190-199': 0.4,
 '200-209': 8.799999999999999,
 '210-219': 36.6,
 '220-229': 34.599999999999994,
 '230-239': 8.9,
 '240': 0.7000000000000001}

In [28]:
tweets_so_far

155

In [56]:
def kelly(odds, prices):
    df = pd.DataFrame([odds,prices])
    df = df.T
    df.columns = ["odds", "prices"]
    df["odds"] = df["odds"] * .01
    df["win"] = 1 - df["prices"]
    df["b"] = df["win"] / df["prices"]
    df["q"] = 1 - df["odds"]
    df["f"] = df["odds"] - df["q"] / df["b"]
    return df
    
t = kelly(odds, prices)

In [57]:
t

,odds,prices,win,b,q,f
169,0.000,0.01,0.99,99.000000,1.000,-0.010101
170-179,0.000,0.02,0.98,49.000000,1.000,-0.020408
180-189,0.000,0.05,0.95,19.000000,1.000,-0.052632
190-199,0.004,0.10,0.90,9.000000,0.996,-0.106667
200-209,0.088,0.15,0.85,5.666667,0.912,-0.072941
210-219,0.366,0.18,0.82,4.555556,0.634,0.226829
220-229,0.346,0.16,0.84,5.250000,0.654,0.221429
230-239,0.089,0.15,0.85,5.666667,0.911,-0.071765
240,0.007,0.38,0.62,1.631579,0.993,-0.601613
